In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import folium
from folium.plugins import HeatMap
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

from sklearn.metrics import mean_squared_error

from pandas import DataFrame

import matplotlib.pyplot as plt

# #evaluation = pd.DataFrame({'Model': [],
#                            'Details':[],
#                            'Root Mean Squared Error (RMSE)':[],
#                            'R-squared (training)':[],
#                            'Adjusted R-squared (training)':[],
#                            'R-squared (test)':[],
#                            'Adjusted R-squared (test)':[],
#                            '5-Fold Cross Validation':[]})
                           
all_datadf=pd.read_csv('good_dataset.csv')




/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
#remove outliers function
def remove_outliers(df,columns,n_std):
    for col in columns:
        #print('Working on column: {}'.format(col))
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
    print(str(round(mean,2)) + ', ' +  str(round(sd,2)))
    #print(df[col] <= mean+(n_std*sd))    
    return df

In [3]:
#basic cleaning

all_datadf=all_datadf.fillna(0)
all_datadf=all_datadf.iloc[:, 2:]
all_datadf=all_datadf.drop_duplicates()

#price cleaning, drop 0s
all_datadf['price']=pd.to_numeric(all_datadf['price'], errors='coerce')
all_datadf=all_datadf[all_datadf['price'] > 0]


#numbedrooms cleaning
all_datadf=all_datadf[all_datadf['numberOfBedrooms'] < 10]
all_datadf=all_datadf[all_datadf['surfaceAreaOfPlot'] < 15000]




In [4]:
#epc, state and property type to continuous
#energy type fill na

epc_dict={'A++': 9, 'A+': 8, 'A' : 7, 'B_A+': 6,'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1, 'G_D' : 1, 'G_B' : 1, 'E_D': 2}

state_dict={'TO_RESTORE' : 1, 'TO_RENOVATE' : 2, 'GOOD' : 3, 'TO_BE_DONE_UP': '4', 'JUST_RENOVATED' : 5, 'AS_NEW' : 6}

property_type_dict={'HOUSE' : 1, 'APARTMENT': 0}

energy_type_dict={'0': 'not_specified'}

all_datadf=all_datadf.replace(epc_dict)
all_datadf=all_datadf.replace(state_dict)
all_datadf=all_datadf.replace(property_type_dict)
all_datadf['energyHeatingType']=all_datadf['energyHeatingType'].replace(energy_type_dict)

all_datadf['epcScore']=pd.to_numeric(all_datadf['epcScore'], errors='coerce')
all_datadf['stateOfBuilding']=pd.to_numeric(all_datadf['stateOfBuilding'], errors='coerce')


In [5]:
#postal codes in blocks til 50

all_datadf['postalCode']=all_datadf['postalCode'] // 50 * 50
postalCodes=all_datadf['postalCode'].unique()
#postalCodesdf=pd.DataFrame(postalCodes, columns=['postalCodes'])
all_datadf['postalCode']=all_datadf['postalCode'].astype('string')



In [6]:

from statistics import median

#most interesting dataframe
bestdf=all_datadf[['price', 'stateOfBuilding', 'epcScore', 'postalCode', 'province', 'typeOfProperty', 'numberOfBedrooms', 'livableArea', 'surfaceAreaOfPlot']]





In [ ]:
outlier_columns=all_datadf[['price', 'livableArea', 'surfaceAreaOfPlot']]

bestdf=remove_outliers(bestdf, outlier_columns, 1)

bestdf.describe()

In [7]:
#group houses and appts to see distribution of data
besthousedf=bestdf[bestdf['typeOfProperty'] == 1]
bestapptdf=bestdf[bestdf['typeOfProperty'] == 0]
descriptionhouse=besthousedf.describe()
descriptionappt=bestapptdf.describe()

In [8]:
#check 0 count
num0s = bestdf.groupby('typeOfProperty').agg(lambda x: x.eq(0).sum())

#median and means per house and appt category
mediandf=bestdf.groupby('typeOfProperty').agg(lambda x: x.median())
meandf=bestdf.groupby('typeOfProperty').agg(lambda x: x.mean())

#medians from state and epc
medianstatehouse=mediandf.iloc[1][2]
medianstateappt=mediandf.iloc[0][2]
medianepchouse=mediandf.iloc[1][3]
medianepcappt=mediandf.iloc[0][3]





In [9]:
#fill in state and epcs with medians
besthousedf['epcScore']=besthousedf['epcScore'].apply(lambda x: medianepchouse if x == 0 else x)
besthousedf['stateOfBuilding']=besthousedf['stateOfBuilding'].apply(lambda x: medianstatehouse if x == 0 else x)


bestapptdf['epcScore']=bestapptdf['epcScore'].apply(lambda x: medianepcappt if x == 0 else x)
bestapptdf['stateOfBuilding']=bestapptdf['stateOfBuilding'].apply(lambda x: medianstatehouse if x == 0 else x)

descriptionhouse=besthousedf.describe()
descriptionappt=bestapptdf.describe()



In [10]:
label_col = 'price'
cat_cols = bestdf [[
    'province',
    'postalCode',
    #'energyHeatingType'
]]

num_cols = bestdf[[
    'typeOfProperty',
    'epcScore',
    'numberOfBedrooms',
    'livableArea',
    'stateOfBuilding'
]]

print('number of numerical features: ' + (num_cols.columns))
print('number of categorical features: ', + (cat_cols.columns))

bestdf.dtypes

Index(['number of numerical features: typeOfProperty',
       'number of numerical features: epcScore',
       'number of numerical features: numberOfBedrooms',
       'number of numerical features: livableArea',
       'number of numerical features: stateOfBuilding'],
      dtype='object')
number of categorical features:  Index(['province', 'postalCode'], dtype='object')


price                float64
stateOfBuilding        int64
epcScore               int64
postalCode            string
province              object
typeOfProperty         int64
numberOfBedrooms       int64
livableArea          float64
surfaceAreaOfPlot    float64
dtype: object

In [11]:
#here choose for house or appt, besthousedf or bestapptdf

y_col='price'
y=besthousedf[y_col]
X=(besthousedf[bestapptdf.columns.drop(y_col)]).reset_index(drop=True)

In [12]:
enc=OneHotEncoder(handle_unknown='ignore', sparse=False)

prov_encodeddf=pd.DataFrame(enc.fit_transform(X[['province']]), columns=enc.get_feature_names()).reset_index(drop=True)
postal_encodeddf=pd.DataFrame(enc.fit_transform(X[['postalCode']]), columns=enc.get_feature_names()).reset_index(drop=True)

encodedX=X[['stateOfBuilding', 'numberOfBedrooms', 'surfaceAreaOfPlot', 'livableArea']].join(prov_encodeddf)
encodedX=encodedX.join(postal_encodeddf)
X=encodedX


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()

X_train[['surfaceAreaOfPlot', 'livableArea']]=scaler.fit_transform(X_train[['surfaceAreaOfPlot', 'livableArea']])
X_test[['surfaceAreaOfPlot', 'livableArea']]=scaler.fit_transform(X_test[['surfaceAreaOfPlot', 'livableArea']])


In [26]:

# Xtreme Gradient Boosting (XGBoost)
from xgboost import XGBRegressor
RegModel=XGBRegressor(max_depth=2, 
                      learning_rate=0.1, 
                      n_estimators=1000, 
                      objective='reg:linear', 
                      booster='gbtree',
                      random_state=12)

# Printing all the parameters of XGBoost
print(RegModel)

# Creating the model on Training Data
XGB=RegModel.fit(X_train,y_train)
prediction=XGB.predict(X_test)

from sklearn import metrics
# Measuring Goodness of fit in Training data
print('R2 Value train:', metrics.r2_score(y_train, XGB.predict(X_train)))


print('R2 test:', metrics.r2_score(y_test, prediction))
print(metrics.mean_absolute_error(y_test,prediction))

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=2,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', predictor=None,
             random_state=12, reg_alpha=None, ...)
[13:26:36] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
R2 Value: 0.8479575376989233
0.6957521847376384
117624.16040789377
